**Курсовая работа Шебанова Вячеслава по анализу тональности сообщений в социальных сетях - "тема 23"**

*Блок с установкой всех зависимостей для работы* 

In [2]:
import sys
import subprocess

def install_package(package):
    """Функция для установки пакета через pip"""
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", package])

print("Устанавливаю необходимые библиотеки... (может занять 2-3 минуты)")

packages = [
    "pandas",
    "numpy",
    "matplotlib",
    "seaborn",
    "scikit-learn",
    "xgboost",
    "catboost",
    "lightgbm",
    "scipy"
]

for pkg in packages:
    try:
        __import__(pkg.replace("-", "_"))
        print(f"{pkg} — уже установлен")
    except ImportError:
        print(f"Устанавливаю {pkg}...")
        install_package(pkg)

print("\nВсе зависимости успешно установлены!\n")


Устанавливаю необходимые библиотеки... (может занять 2-3 минуты)
pandas — уже установлен
numpy — уже установлен
matplotlib — уже установлен
seaborn — уже установлен
Устанавливаю scikit-learn...
xgboost — уже установлен
catboost — уже установлен
Устанавливаю lightgbm...
scipy — уже установлен

Все зависимости успешно установлены!



*Импорт необходимых библиотек*

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
from scipy.sparse import hstack

# Машинное обучение
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, StackingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

# Метрики
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, roc_auc_score

# ДОБАВЛЯЕМ ЭТОТ ИМПОРТ — ИСПРАВЛЕНИЕ ОШИБКИ!
import sklearn
import xgboost
import catboost
import lightgbm

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
sns.set(style="whitegrid", font_scale=1.1)
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = (10, 6)

*Загрузка данных*